In [27]:
%matplotlib inline
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pygeocoder import Geocoder
# from citipy import citipy

# Google developer API key
from config import gkey

In [51]:
# File to Load
data_to_load = "data/Meteorite_Landings.csv"
meteor_landings = pd.read_csv(data_to_load, parse_dates=True)

# Display the data table for preview
meteor_landings.head()

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,States,Counties
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)",NaN,NaN
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)",NaN,NaN
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)",NaN,NaN
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)",NaN,NaN
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)",NaN,NaN


In [110]:
# Refine source data into a starting DataFrame
meteor_landings_df = meteor_landings.filter(['name','mass (g)','year','reclat','reclong'], axis=1)
meteor_landings_df = meteor_landings_df.rename(columns={'mass (g)': 'mass',
                                                        'year': 'date',
                                                        'reclat': 'latitude',
                                                        'reclong': 'longitude'})
print('Before: ', len(meteor_landings_df))
meteor_landings_df = meteor_landings_df.dropna(subset=['date'])
print('After: ', len(meteor_landings_df))

meteor_landings_df['state'] = ''
meteor_landings_df['country'] = ''

meteor_landings_df.head()

Before:  45716
After:  45425


,name,mass,date,latitude,longitude,state,country
0,Aachen,21.0,01/01/1880 12:00:00 AM,50.77500,6.08333,,
1,Aarhus,720.0,01/01/1951 12:00:00 AM,56.18333,10.23333,,
2,Abee,107000.0,01/01/1952 12:00:00 AM,54.21667,-113.00000,,
3,Acapulco,1914.0,01/01/1976 12:00:00 AM,16.88333,-99.90000,,
4,Achiras,780.0,01/01/1902 12:00:00 AM,-33.16667,-64.95000,,


In [111]:
meteor_landings_df['date'] = pd.to_datetime(meteor_landings_df['date'], errors='coerce').dt.year

print('Before: ', len(meteor_landings_df))
meteor_landings_df = meteor_landings_df.dropna(subset=['date'])
print('After: ', len(meteor_landings_df))

meteor_landings_df.head()

Before:  45425
After:  45404


,name,mass,date,latitude,longitude,state,country
0,Aachen,21.0,1880.0,50.77500,6.08333,,
1,Aarhus,720.0,1951.0,56.18333,10.23333,,
2,Abee,107000.0,1952.0,54.21667,-113.00000,,
3,Acapulco,1914.0,1976.0,16.88333,-99.90000,,
4,Achiras,780.0,1902.0,-33.16667,-64.95000,,


In [107]:
keyed_geocoder = Geocoder(api_key = gkey)

In [112]:
# Use Google Geocoding API to reverse lookup lat & long to get country and state

for row in range(10):
    results = keyed_geocoder.reverse_geocode(meteor_landings_df['latitude'][row],
                                             meteor_landings_df['longitude'][row])
    meteor_landings_df.loc[row,'state'] = results.administrative_area_level_1
    meteor_landings_df.loc[row,'country'] = results.country

meteor_landings_df.head(15)

,name,mass,date,latitude,longitude,state,country
0,Aachen,21.0,1880.0,50.77500,6.08333,Nordrhein-Westfalen,Germany
1,Aarhus,720.0,1951.0,56.18333,10.23333,None,Denmark
2,Abee,107000.0,1952.0,54.21667,-113.00000,Alberta,Canada
3,Acapulco,1914.0,1976.0,16.88333,-99.90000,Guerrero,Mexico
4,Achiras,780.0,1902.0,-33.16667,-64.95000,Córdoba,Argentina
5,Adhi Kot,4239.0,1919.0,32.10000,71.80000,Punjab,Pakistan
6,Adzhi-Bogdo (stone),910.0,1949.0,44.83333,95.16667,Govi-Altai,Mongolia
7,Agen,30000.0,1814.0,44.21667,0.61667,Nouvelle-Aquitaine,France
8,Aguada,1620.0,1930.0,-31.60000,-65.23333,Cordoba,Argentina
9,Aguila Blanca,1440.0,1920.0,-30.86667,-64.55000,Córdoba,Argentina


In [76]:
# Group the data by year for sum of landings


In [ ]:
# Group the data by year, sub-group mass

In [ ]:
# Use lat & long to get state using Google Map API
# Add state to new column named "us_state"
# Got this code from 'stack overflow'
# https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city

def getplace(lat, lon):
    url = "http://maps.googleapis.com/maps/api/geocode/json?"
    url += "latlng=%s,%s&sensor=false" % (lat, lon)
    v = urlopen(url).read()
    j = json.loads(v)
    components = j['results'][0]['address_components']
    country = town = None
    for c in components:
        if "country" in c['types']:
            country = c['long_name']
        if "postal_town" in c['types']:
            town = c['long_name']
    return town, country

In [ ]:
# The website below shows how to do what we need without Google API
# https://chrisalbon.com/python/data_wrangling/geocoding_and_reverse_geocoding/



In [ ]:
# Overlay state sum of landings over map

In [ ]:
# Find out which state had the most landings

In [ ]:
# Find out how many landings happened within X-miles of a city in that state